**Data Preparation**

Preprocess the images:

In [ ]:
import cv2
import numpy as np
import os

def load_images_and_labels(data_dir):
    images = []
    ages = []
    genders = []
    for filename in os.listdir(data_dir):
            total = data_dir + "/" + filename
            print(total)
            img = cv2.imread(total)
            img = cv2.resize(img, (200, 200))
            images.append(img)
            age=int(filename.split('_')[0])
            gender=int(filename.split('_')[1])
            ages.append(age)
            genders.append(gender)
    return np.array(images), np.array(ages), np.array(genders)

images, ages, genders = load_images_and_labels('/content/UTKFace')


Split the dataset:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_age, y_test_age, y_train_gender, y_test_gender = train_test_split(
    images, ages, genders, test_size=0.2, random_state=42)

**Model Building**

Build the model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_layer = Input(shape=(200, 200, 3))

x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)

age_output = Dense(1, name='age_output')(x)
gender_output = Dense(1, activation='sigmoid', name='gender_output')(x)

model = Model(inputs=input_layer, outputs=[age_output, gender_output])

model.compile(optimizer='adam',
              loss={'age_output': 'mean_squared_error', 'gender_output': 'binary_crossentropy'},
              metrics={'age_output': 'mae', 'gender_output': 'accuracy'})

**Model Training**

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/visualise activation maps/age_gender_model.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=50,
    verbose=1,
    restore_best_weights=True
)


Train the model

In [ ]:
history = model.fit(
    X_train,
    {'age_output': y_train_age, 'gender_output': y_train_gender},
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[checkpoint, early_stopping]
)


**Evaluation**

Evaluate the model

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/visualise activation maps/age_gender_model.h5')

In [ ]:
results = model.evaluate(X_test, {'age_output': y_test_age, 'gender_output': y_test_gender})
print(f"Test Loss, Test Accuracy: {results}")